In [1]:
%pylab inline

import pandas as pd
import pickle as pk
from scipy import sparse as sp
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, LancasterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import *

# p_df = pd.read_csv('../data/HappyDB/happydb/data/original_hm_test.csv')
p_df = pd.read_csv('../data/HappyDB/happydb/data/cleaned_hm.csv')

# stemmer = LancasterStemmer()
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS:
            result.append(lemmatize_stemming(token))
    return result

p_docs = p_df['cleaned_hm'].map(preprocess)
f_docs = []
for elem in p_docs:
    f_docs.append(elem)

Populating the interactive namespace from numpy and matplotlib


In [2]:
from gensim.models import Phrases
# Add bigrams and trigrams to docs (only ones that appear 10 times or more).
bigram = Phrases(f_docs, min_count=10)
trigram = Phrases(bigram[f_docs])

for idx in range(len(f_docs)):
    for token in bigram[f_docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            f_docs[idx].append(token)
    for token in trigram[f_docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            f_docs[idx].append(token)

/home/zhida/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [3]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(f_docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))

corpus = [dictionary.doc2bow(doc) for doc in f_docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique words in initital documents: 23288
Number of unique words after removing rare and common words: 6780
Number of unique tokens: 6780
Number of documents: 100535


In [4]:
from gensim.models import LdaModel

# Set training parameters.
num_topics = 20
chunksize = 500 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

/home/zhida/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


CPU times: user 37min 6s, sys: 26.1 s, total: 37min 32s
Wall time: 9min 23s


In [5]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

pyLDAvis.gensim.prepare(model, corpus, dictionary)

/home/zhida/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.329236 -0.048971       1        1  15.607309
9      0.030690  0.144311       2        1   6.677814
8      0.039971  0.027360       3        1   6.615011
5      0.058658  0.017092       4        1   5.938020
6     -0.077203 -0.000333       5        1   5.120809
2     -0.306958 -0.051176       6        1   5.111130
19    -0.045068  0.099687       7        1   4.625513
12     0.088286 -0.298313       8        1   4.580343
10     0.031939  0.011251       9        1   4.288637
16     0.031436  0.011150      10        1   4.159392
4      0.047278  0.152296      11        1   4.155274
18     0.033031  0.011377      12        1   4.129020
17     0.053841  0.022664      13        1   4.024023
14     0.089235 -0.294832      14        1   3.735957
1      0.039864  0.025786      15        1   3.713385
11     0.038624  0.017983      16        1   3.702353
13     0.039979  0.028609      17        1   3.591399
7      0.055247  0.086609      18        1   3.464792
3      0.046804  0.025799      19        1   3.428880
15     0.033582  0.011649      20        1   3.330930, topic_info=     Category          Freq             Term         Total  loglift  logprob
term                                                                        
3     Default  12501.000000               go  12501.000000  30.0000  30.0000
7     Default  17868.000000            happy  17868.000000  29.0000  29.0000
6     Default  15535.000000              get  15535.000000  28.0000  28.0000
27    Default   9164.000000              new   9164.000000  27.0000  27.0000
248   Default  12056.000000             work  12056.000000  26.0000  26.0000
141   Default  11436.000000             time  11436.000000  25.0000  25.0000
92    Default   8934.000000              day   8934.000000  24.0000  24.0000
44    Default   5028.000000            watch   5028.000000  23.0000  23.0000
24    Default   5416.000000            night   5416.000000  22.0000  22.0000
93    Default   5896.000000           friend   5896.000000  21.0000  21.0000
26    Default   5641.000000             come   5641.000000  20.0000  20.0000
90    Default   4343.000000             play   4343.000000  19.0000  19.0000
114   Default   7246.000000            today   7246.000000  18.0000  18.0000
37    Default   4012.000000            enjoy   4012.000000  17.0000  17.0000
14    Default   4752.000000          friends   4752.000000  16.0000  16.0000
177   Default   4739.000000             home   4739.000000  15.0000  15.0000
106   Default   3832.000000           family   3832.000000  14.0000  14.0000
232   Default   4180.000000           dinner   4180.000000  13.0000  13.0000
848   Default   3896.000000             life   3896.000000  12.0000  12.0000
15    Default   6596.000000             good   6596.000000  11.0000  11.0000
53    Default   4248.000000             feel   4248.000000  10.0000  10.0000
273   Default   3561.000000         favorite   3561.000000   9.0000   9.0000
364   Default   3528.000000             take   3528.000000   8.0000   8.0000
9     Default   3863.000000              son   3863.000000   7.0000   7.0000
180   Default   4204.000000        come_home   4204.000000   6.0000   6.0000
295   Default   3562.000000              buy   3562.000000   5.0000   5.0000
469   Default   3408.000000  happiest_moment   3408.000000   4.0000   4.0000
553   Default   3518.000000              old   3518.000000   3.0000   3.0000
176   Default   3848.000000         daughter   3848.000000   2.0000   2.0000
39    Default   5605.000000        yesterday   5605.000000   1.0000   1.0000
...       ...           ...              ...           ...      ...      ...
226   Topic20   2124.621094              win   2124.775391   3.4018  -2.6987
69    Topic20   1468.178589             hear   1468.331909   3.4018  -3.0683
208   Topic20   1452.388184              kid   1452.541992   3.4018  -3.07

In [6]:
%pylab inline

import pandas as pd
import pickle as pk
from scipy import sparse as sp
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, LancasterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import *

# p_df = pd.read_csv('../data/HappyDB/happydb/data/original_hm_test.csv')
p_df = pd.read_csv('../data/HappyDB/happydb/data/cleaned_hm.csv')

# stemmer = LancasterStemmer()
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS:
            result.append(lemmatize_stemming(token))
    return result

p_docs = p_df['cleaned_hm'].map(preprocess)
f_docs = []
for elem in p_docs:
    f_docs.append(elem)

Populating the interactive namespace from numpy and matplotlib


In [7]:
from gensim.models import Phrases
# Add bigrams and trigrams to docs (only ones that appear 10 times or more).
bigram = Phrases(f_docs, min_count=10)
trigram = Phrases(bigram[f_docs])

for idx in range(len(f_docs)):
    for token in bigram[f_docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            f_docs[idx].append(token)
    for token in trigram[f_docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            f_docs[idx].append(token)

/home/zhida/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [8]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(f_docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))

corpus = [dictionary.doc2bow(doc) for doc in f_docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique words in initital documents: 23288
Number of unique words after removing rare and common words: 6780
Number of unique tokens: 6780
Number of documents: 100535


In [9]:
from gensim.models import LdaModel

# Set training parameters.
num_topics = 20
chunksize = 500 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

/home/zhida/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


CPU times: user 10min 11s, sys: 68.3 ms, total: 10min 11s
Wall time: 10min 11s


In [10]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

pyLDAvis.gensim.prepare(model, corpus, dictionary)

/home/zhida/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
12     0.062704 -0.222474       1        1  9.965510
4      0.060817  0.135005       2        1  9.315540
15     0.032703 -0.042536       3        1  8.408925
14     0.019415 -0.037206       4        1  5.953207
3      0.038749 -0.017800       5        1  5.602638
6     -0.334482  0.024652       6        1  4.978908
10    -0.277514  0.039415       7        1  4.919145
17     0.080597  0.268969       8        1  4.512018
7      0.032589 -0.040811       9        1  4.274847
11    -0.180520 -0.006880      10        1  4.192221
1      0.068036 -0.200159      11        1  4.151354
16     0.034575 -0.024321      12        1  4.044891
8      0.064165  0.080250      13        1  3.914734
18     0.031924 -0.038329      14        1  3.903692
0      0.031951 -0.038842      15        1  3.884209
2      0.039897 -0.019617      16        1  3.872385
9      0.031758 -0.036382      17        1  3.759536
13     0.037430 -0.011589      18        1  3.538665
19     0.030785 -0.038795      19        1  3.535216
5      0.094423  0.227450      20        1  3.272357, topic_info=     Category          Freq               Term         Total  loglift  logprob
term                                                                          
7     Default  18007.000000              happy  18007.000000  30.0000  30.0000
6     Default  15438.000000                get  15438.000000  29.0000  29.0000
141   Default  11405.000000               time  11405.000000  28.0000  28.0000
248   Default  11980.000000               work  11980.000000  27.0000  27.0000
27    Default   9855.000000                new   9855.000000  26.0000  26.0000
3     Default  11384.000000                 go  11384.000000  25.0000  25.0000
92    Default   9003.000000                day   9003.000000  24.0000  24.0000
26    Default   5486.000000               come   5486.000000  23.0000  23.0000
114   Default   7201.000000              today   7201.000000  22.0000  22.0000
93    Default   6645.000000             friend   6645.000000  21.0000  21.0000
24    Default   5104.000000              night   5104.000000  20.0000  20.0000
44    Default   4982.000000              watch   4982.000000  19.0000  19.0000
15    Default   6555.000000               good   6555.000000  18.0000  18.0000
177   Default   4609.000000               home   4609.000000  17.0000  17.0000
90    Default   4456.000000               play   4456.000000  16.0000  16.0000
39    Default   5649.000000          yesterday   5649.000000  15.0000  15.0000
180   Default   4089.000000          come_home   4089.000000  14.0000  14.0000
848   Default   3934.000000               life   3934.000000  13.0000  13.0000
143   Default   5280.000000               able   5280.000000  12.0000  12.0000
176   Default   3551.000000           daughter   3551.000000  11.0000  11.0000
9     Default   3893.000000                son   3893.000000  10.0000  10.0000
295   Default   3830.000000                buy   3830.000000   9.0000   9.0000
232   Default   4432.000000             dinner   4432.000000   8.0000   8.0000
14    Default   4327.000000            friends   4327.000000   7.0000   7.0000
469   Default   3298.000000    happiest_moment   3298.000000   6.0000   6.0000
165   Default   3446.000000               give   3446.000000   5.0000   5.0000
46    Default   3323.000000               game   3323.000000   4.0000   4.0000
37    Default   4088.000000              enjoy   4088.000000   3.0000   3.0000
106   Default   3904.000000             family   3904.000000   2.0000   2.0000
139   Default   3032.000000          long_time   3032.000000   1.0000   1.0000
...       ...           ...                ...           ...      ...      ...
400   Topic20    897.012756               lose    897.165405   3.4195  -3.5431
2079  Topic20    809.489807      great_workout    809.640198   3.4195  -3.6458
905   Topic20    818.90564